### MONSIEUR DANS CE CODE YA LES MEMES ETAPES SAUF QUE DANS CE CODE JAI FAIS UN AUTRE TYPE DEMBEDDING QUI EST BGE-M3 POUR TESTER LES PERFORMANCES !

In [1]:
import pandas as pd

data=pd.read_csv("fusionned_file.csv",sep=";")
data.head()


,source sentence,simplified sentence,run_id,snt_id,No error,Random generation,Syntax error,Contradiction,Simple punctuation / grammar errors,Redundancy,Format misalignement,Prompt misalignement,Out-of-Scope Generation,Topic shift,Oversimplification of Logical Arguments,Overgeneralization,Loss of Informative Content,Factuality hallucination,Faithfulness hallucination,Commentaire
0,":[0],""non-coding RNAs (lncRNAs) may play crit...","Long non-coding RNAs, or lncRNAs, may play si...",UBO_Task3.1_Phi4mini-sl,G04.2_2519200480_1,VRAI,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN
1,"Recently, lncRNAs have been widely identified ...",Small RNA molecules are found in complex cell...,UBO_Task3.1_Phi4mini-sl,G04.2_2519200480_2,VRAI,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN
2,"However, there is rare information about them ...",There is unusual information here that doesn'...,UBO_Task3.1_Phi4mini-sl,G04.2_2519200480_3,FAUX,False,False,False,False,False,False,False,True,True,False,False,False,False,False,NaN
3,"During humanitarian crises, there is a need fo...","When crises happen, people need quick and larg...",UBO_Task3.1_Phi4mini-sl,G07.1_3007380484_1,FAUX,False,False,False,False,False,False,False,True,False,False,False,False,False,False,NaN
4,Such need creates the base for misinformation ...,This reduction helps in combating climate chan...,UBO_Task3.1_Phi4mini-sl,G07.1_3007380484_2,FAUX,False,False,False,False,True,False,True,False,True,False,False,False,False,False,NaN


In [2]:
data.drop(columns=['run_id', 'snt_id','No error','Syntax error','Loss of Informative Content','Random generation','Simple punctuation / grammar errors','Redundancy','Contradiction','Format misalignement','Prompt misalignement','Topic shift',	'Oversimplification of Logical Arguments',	'Overgeneralization','Factuality hallucination',	'Faithfulness hallucination',	'Commentaire'], inplace=True)

In [3]:
data.head(1)

,source sentence,simplified sentence,Out-of-Scope Generation
0,":[0],""non-coding RNAs (lncRNAs) may play crit...","Long non-coding RNAs, or lncRNAs, may play si...",False


In [4]:
bool_columns = data.columns[data.dtypes == 'bool']

# Calculer les pourcentages de TRUE et FALSE pour chaque colonne booléenne
percentages = {
    col: {
        'TRUE (%)': data[col].mean() * 100,
        'FALSE (%)': (1 - data[col].mean()) * 100
    }
    for col in bool_columns
}

# Convertir en DataFrame pour afficher les résultats
result_df = pd.DataFrame(percentages).T

# Afficher les résultats
#import ace_tools as tools; tools.display_dataframe_to_user(name="Pourcentage des Colonnes Booléennes", dataframe=result_df)
print(result_df)

                         FALSE (%)   TRUE (%)
Out-of-Scope Generation  86.060606  13.939394


## Preprocessing before embeddings

In [6]:
import pandas as pd
import re

# Fonction de nettoyage
def clean_text(text):
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Supprime caractères spéciaux (, . () [])
    text = text.lower().strip()  # Convertit en minuscules et supprime les espaces inutiles
    return text

data['source sentence'] = data['source sentence'].apply(clean_text)
data['simplified sentence'] = data['simplified sentence'].apply(clean_text)
print(data.head())


                                     source sentence  \
0  0noncoding rnas lncrnas may play critical role...   
1  recently lncrnas have been widely identified a...   
2  however there is rare information about them i...   
3  during humanitarian crises there is a need for...   
4  such need creates the base for misinformation ...   

                                 simplified sentence  Out-of-Scope Generation  
0  long noncoding rnas or lncrnas may play signif...                    False  
1  small rna molecules are found in complex cells...                    False  
2  there is unusual information here that doesnt ...                     True  
3  when crises happen people need quick and large...                     True  
4  this reduction helps in combating climate chan...                    False  


### Applying  Embeddings : HUGGING FACE  bge-m3!!!!

Le modèle **`BAAI/bge-m3`** (Base General Embedding - M3) de **Hugging Face** est un modèle de génération d'embeddings textuels polyvalent et optimisé pour une variété de tâches NLP. Voici ses principales caractéristiques :

### **Description du modèle BGE-M3 :**
1. **Objectif** :
   - Conçu pour produire des embeddings vectoriels riches et sémantiquement significatifs à partir de textes.
   - Optimisé pour des tâches telles que :
     - Recherche sémantique.
     - Classification.
     - Correspondance entre documents.

2. **Architecture** :
   - Basé sur un modèle transformer (probablement dérivé de BERT ou similaire), avec des ajustements spécifiques pour maximiser la qualité des embeddings.

3. **Points forts** :
   - **Compression efficace** : Les vecteurs générés par le modèle sont denses et bien adaptés pour des comparaisons dans un espace latent.
   - **Polyvalence** : Convient à une large gamme de langues et de domaines grâce à un entraînement sur des corpus diversifiés.
   - **Optimisation GPU (FP16)** : Compatible avec les calculs en demi-précision (FP16), ce qui améliore les performances sur des machines équipées de GPU.

4. **Avantages pour vos données** :
   - Permet de capturer des nuances sémantiques fines entre `source sentence` et `simplified sentence`.
   - Génère des embeddings directement utilisables pour des tâches de classification, en réduisant la dépendance à des prétraitements supplémentaires.

---

### **Pourquoi le BGE-M3 a donné de meilleurs résultats ?**
1. **Qualité des embeddings** :
   - Le modèle capture les relations sémantiques complexes mieux que des modèles généralistes comme `all-MiniLM-L6-v2` (Sentence-BERT).

2. **Alignement sémantique** :
   - Idéal pour comparer des phrases similaires mais avec des différences significatives dans les informations.

3. **Adaptation aux tâches** :
   - Entraîné sur des objectifs adaptés à des cas comme la recherche et la classification, ce qui correspond à votre besoin.

---


In [7]:
from FlagEmbedding import BGEM3FlagModel
import pandas as pd
import numpy as np

# Charger le modèle BGEM3FlagModel
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

# Générer les embeddings pour chaque colonne en utilisant BGEM3FlagModel
data['source_embedding'] = data['source sentence'].apply(lambda x: model.encode(x))
data['simplified_embedding'] = data['simplified sentence'].apply(lambda x: model.encode(x))

# Conversion des embeddings en matrice
source_embeddings = np.vstack(data['source_embedding'].values)
simplified_embeddings = np.vstack(data['simplified_embedding'].values)

# Ajouter les vecteurs d'embeddings directement au DataFrame
data['source_latent'] = data['source_embedding']
data['simplified_latent'] = data['simplified_embedding']

# Afficher les données avec les colonnes embeddings
print(data.head())

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

...

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


                                     source sentence  \
0  0noncoding rnas lncrnas may play critical role...   
1  recently lncrnas have been widely identified a...   
2  however there is rare information about them i...   
3  during humanitarian crises there is a need for...   
4  such need creates the base for misinformation ...   

                                 simplified sentence  Out-of-Scope Generation  \
0  long noncoding rnas or lncrnas may play signif...                    False   
1  small rna molecules are found in complex cells...                    False   
2  there is unusual information here that doesnt ...                     True   
3  when crises happen people need quick and large...                     True   
4  this reduction helps in combating climate chan...                    False   

                                    source_embedding  \
0  {'dense_vecs': [-0.03040701, 0.020827428, -0.0...   
1  {'dense_vecs': [-0.037408523, 0.016041253, -0....   
2  {'den

In [21]:
data.head(1)

,Out-of-Scope Generation,source_latent,simplified_latent
0,False,"{'dense_vecs': [-0.03040701, 0.020827428, -0.0...","{'dense_vecs': [-0.061536856, 0.0012286099, -0..."


In [22]:
import numpy as np

# Extraire les vecteurs denses des colonnes d'embeddings
data['source_embedding'] = data['source_latent'].apply(lambda x: np.array(x['dense_vecs']))
data['simplified_embedding'] = data['simplified_latent'].apply(lambda x: np.array(x['dense_vecs']))


In [26]:
data.head()

,Out-of-Scope Generation,source_latent,simplified_latent,source_embedding,simplified_embedding,similarity
0,False,"{'dense_vecs': [-0.03040701, 0.020827428, -0.0...","{'dense_vecs': [-0.061536856, 0.0012286099, -0...","[-0.03040701, 0.020827428, -0.024075337, 0.066...","[-0.061536856, 0.0012286099, -0.028455336, 0.0...",0.819644
1,False,"{'dense_vecs': [-0.037408523, 0.016041253, -0....","{'dense_vecs': [-0.03323403, 0.009967967, -0.0...","[-0.037408523, 0.016041253, -0.04064109, 0.018...","[-0.03323403, 0.009967967, -0.016377063, 0.017...",0.617392
2,True,"{'dense_vecs': [-0.03741509, 0.04921532, -0.02...","{'dense_vecs': [-0.027620194, 0.03375707, -0.0...","[-0.03741509, 0.04921532, -0.028045332, -0.008...","[-0.027620194, 0.03375707, -0.03720918, 0.0391...",0.679275
3,True,"{'dense_vecs': [0.017227065, 0.002789266, -0.0...","{'dense_vecs': [0.0112857325, 0.020102756, -0....","[0.017227065, 0.002789266, -0.043252207, 0.024...","[0.0112857325, 0.020102756, -0.055427544, 0.02...",0.716800
4,False,"{'dense_vecs': [0.00042183953, 0.011622541, -0...","{'dense_vecs': [-0.04483482, 0.012996589, -0.0...","[0.00042183953, 0.011622541, -0.046557628, 0.0...","[-0.04483482, 0.012996589, -0.009399194, 0.012...",0.417316


### PCA 

In [74]:
# # Réduction de dimension à 50 via PCA
# pca = PCA(n_components=50, random_state=42)
# source_latent = pca.fit_transform(source_embeddings)
# simplified_latent = pca.transform(simplified_embeddings)

# # Ajouter les vecteurs réduits au DataFrame
# data['source_latent'] = list(source_latent)
# data['simplified_latent'] = list(simplified_latent)


### FEATURE ENGINNERING 

# Fonction pour calculer la similarité cosinus 

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean

def compare_embeddings(vec1, vec2, metric='cosine'):
    if metric == 'cosine':
        return cosine_similarity([vec1], [vec2])[0][0]
    elif metric == 'euclidean':
        return euclidean(vec1, vec2)

# Ajouter une colonne de similarité au DataFrame
data['similarity'] = data.apply(
    lambda row: compare_embeddings(row['source_embedding'], row['simplified_embedding'], metric='cosine'), axis=1
)

# # Afficher le DataFrame final
# print(data[['source embedding', 'simplified embedding', 'similarity']])




In [30]:
# data.drop(columns=['source sentence','simplified sentence','source_embedding','simplified_embedding'], inplace=True)
data.drop(columns=['source_latent','simplified_latent'], inplace=True)

In [31]:
data.head()

,Out-of-Scope Generation,source_embedding,simplified_embedding,similarity
0,False,"[-0.03040701, 0.020827428, -0.024075337, 0.066...","[-0.061536856, 0.0012286099, -0.028455336, 0.0...",0.819644
1,False,"[-0.037408523, 0.016041253, -0.04064109, 0.018...","[-0.03323403, 0.009967967, -0.016377063, 0.017...",0.617392
2,True,"[-0.03741509, 0.04921532, -0.028045332, -0.008...","[-0.027620194, 0.03375707, -0.03720918, 0.0391...",0.679275
3,True,"[0.017227065, 0.002789266, -0.043252207, 0.024...","[0.0112857325, 0.020102756, -0.055427544, 0.02...",0.716800
4,False,"[0.00042183953, 0.011622541, -0.046557628, 0.0...","[-0.04483482, 0.012996589, -0.009399194, 0.012...",0.417316


### Importing Packages!!

In [32]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC

In [34]:
data.head(1)

,Out-of-Scope Generation,source_embedding,simplified_embedding,similarity
0,False,"[-0.03040701, 0.020827428, -0.024075337, 0.066...","[-0.061536856, 0.0012286099, -0.028455336, 0.0...",0.819644


### SPLIT

In [35]:
# X = data[['source_latent', 'simplified_latent', 'similarity']]
X = data[['source_embedding', 'simplified_embedding']]
y = data['Out-of-Scope Generation'].astype(int)

# Combiner les colonnes d'embeddings en un vecteur unique
def combine_features(row):
    # return np.concatenate([row['source_latent'], row['simplified_latent'], [row['similarity']]])
    return np.concatenate([row['source_embedding'], row['simplified_embedding']])
# Appliquer la combinaison sur toutes les lignes
X_combined = np.array(X.apply(combine_features, axis=1).tolist())

# Diviser en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)


## SMOTE OVERSAMPLING CLASSE MINORITAIRE

In [36]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


## Logistic Regression

In [37]:
logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_train_resampled, y_train_resampled)
y_pred = logistic_model.predict(X_test)


In [38]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["False", "True"]))


Confusion Matrix:
[[50  6]
 [ 2  8]]

Classification Report:
              precision    recall  f1-score   support

       False       0.96      0.89      0.93        56
        True       0.57      0.80      0.67        10

    accuracy                           0.88        66
   macro avg       0.77      0.85      0.80        66
weighted avg       0.90      0.88      0.89        66



In [39]:
print(f"The balanced accuracy of the model is "
      f"{balanced_accuracy_score(y_test, y_pred):.3f}")

The balanced accuracy of the model is 0.846


### Modeling with svm

In [40]:
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_resampled, y_train_resampled)
y_pred = svm_model.predict(X_test)


In [41]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["False", "True"]))

Confusion Matrix:
[[50  6]
 [ 4  6]]

Classification Report:
              precision    recall  f1-score   support

       False       0.93      0.89      0.91        56
        True       0.50      0.60      0.55        10

    accuracy                           0.85        66
   macro avg       0.71      0.75      0.73        66
weighted avg       0.86      0.85      0.85        66



In [42]:
print(f"The balanced accuracy of the model is "
      f"{balanced_accuracy_score(y_test, y_pred):.3f}")

The balanced accuracy of the model is 0.746


### GRID SEARCH FOR BETTER SCORING!

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Définir les hyperparamètres à tester
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

# Initialiser GridSearchCV avec SVM
svm = SVC()
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='f1', n_jobs=-1)

# Entraîner GridSearchCV sur les données
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres et résultats
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

# Entraîner le modèle final avec les meilleurs paramètres
best_svm = grid_search.best_estimator_
y_pred = best_svm.predict(X_test)

# Évaluation
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["False", "True"]))


Best Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Best F1 Score: 0.2833333333333333
Confusion Matrix:
[[55  1]
 [ 6  4]]

Classification Report:
              precision    recall  f1-score   support

       False       0.90      0.98      0.94        56
        True       0.80      0.40      0.53        10

    accuracy                           0.89        66
   macro avg       0.85      0.69      0.74        66
weighted avg       0.89      0.89      0.88        66



In [44]:
print(f"The balanced accuracy of the model is "
      f"{balanced_accuracy_score(y_test, y_pred):.3f}")

The balanced accuracy of the model is 0.691
